# Ungraded Lab: TensorFlow Model Analysis

In production systems, the decision to deploy a model usually goes beyond the global metrics (e.g. accuracy) set during training. It is also important to evaluate how your model performs in different scenarios. For instance, does your weather forecasting model perform equally well in summer compared to winter? Or does your camera-based defect detector work only in certain lighting conditions? This type of investigation helps to ensure that your model can handle different cases. More than that, it can help uncover any learned biases that can result in a negative experience for your users. For example, if you're supposed to have a gender-neutral application, you don't want your model to only work well for one while poorly for another.

In this lab, you will be working with [TensorFlow Model Analysis (TFMA)](https://www.tensorflow.org/tfx/guide/tfma) -- a library built specifically for analyzing a model's performance across different configurations. It allows you to specify slices of your data, then it will compute and visualize how your model performs on each slice. You can also set thresholds that your model must meet before it is marked ready for deployment. These help you make better decisions regarding any improvements you may want to make to boost your model's performance and ensure fairness.

For this exercise, you will use TFMA to analyze models trained on the [Census Income dataset](http://archive.ics.uci.edu/ml/datasets/Census+Income). Specifically, you will:

* study and setup the starter files to use with TFMA
* make a configuration file to tell TFMA what data slices it will analyze and the metrics it will compute
* visualize TFMA's outputs in a notebook environment
* generate a time series of a model's performance
* compare the performance of two models so you can decide which one to push to production

*Credits: Some of the code and discussions are based on the TensorFlow team's [official tutorial](https://www.tensorflow.org/tfx/tutorials/model_analysis/tfma_basic).*

## Setup

In this section, you will first setup your workspace to have all the modules and files to work with TFMA. You will 
* install required libraries, 
* download starter files that will contain the dataset, schema, and pretrained models you will analyze
* prepare the dataset so it can be consumed by TFMA
* observe how the models transform the raw features

### Install Jupyter Extensions
If running in a local Jupyter notebook, then these Jupyter extensions must be installed in the environment before running Jupyter. These are already available in Colab so we'll just leave the commands here for reference.

```bash
jupyter nbextension enable --py widgetsnbextension --sys-prefix 
jupyter nbextension install --py --symlink tensorflow_model_analysis --sys-prefix 
jupyter nbextension enable --py tensorflow_model_analysis --sys-prefix
```

In [1]:
!jupyter nbextension enable --py widgetsnbextension --sys-prefix 
!jupyter nbextension install --py --symlink tensorflow_model_analysis --sys-prefix 
!jupyter nbextension enable --py tensorflow_model_analysis --sys-prefix 

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: OK
Traceback (most recent call last):
  File "/home/tjp1992/.pyenv/versions/3.8.12/envs/coursera/bin/jupyter-nbextension", line 8, in <module>
    sys.exit(main())
  File "/home/tjp1992/.pyenv/versions/3.8.12/envs/coursera/lib/python3.8/site-packages/jupyter_core/application.py", line 269, in launch_instance
    return super().launch_instance(argv=argv, **kwargs)
  File "/home/tjp1992/.pyenv/versions/3.8.12/envs/coursera/lib/python3.8/site-packages/traitlets/config/application.py", line 982, in launch_instance
    app.start()
  File "/home/tjp1992/.pyenv/versions/3.8.12/envs/coursera/lib/python3.8/site-packages/notebook/nbextensions.py", line 972, in start
    super().start()
  File "/home/tjp1992/.pyenv/versions/3.8.12/envs/coursera/lib/python3.8/site-packages/jupyter_core/application.py", line 258, in start
    self.subapp.start()
  File "/home/tjp1992/.pyenv/versions/3.8.12/envs/coursera/lib/python3.8/si

### Install libraries

This will pull in all the dependencies and will take 6 to 8 minutes to complete.


In [2]:
# Upgrade pip to the latest version and install required packages
!pip install -U pip
!pip install --use-deprecated=legacy-resolver tensorflow_data_validation==1.1.0
!pip install --use-deprecated=legacy-resolver tensorflow-transform==1.0.0
!pip install --use-deprecated=legacy-resolver tensorflow-model-analysis==0.32.0
!pip install apache-beam==2.32.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 5.4 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: pip
    Found existing installation: pip 22.2.2
    Uninstalling pip-22.2.2:
      Successfully uninstalled pip-22.2.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 5.0 MB/s eta 0:00:0000:0100:01m
  Using cached pyarrow-2.0.0-cp38-cp38-manylinux2014_x86_64.whl (17.8 MB)
  Using cached joblib-0.14.1-py2.py3-none-any.whl (294 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.0/19.0 MB 3.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.1/48.1 kB 731.4 kB/s eta 0:00:00a 0:00:01
  Using cached numpy-1.19.5-cp38-cp38-manylinux2010_x86_64.whl (14.9 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 189.1/189.1 kB 1.7 MB/s eta 0:00:0000:01
  Using cached dill-0.3.1.1.tar.gz (151 kB)
  Preparing metadata (setup.py) ... done
  Using cached google_cloud_core-1.7.3-py2.py3-none-any.whl (28 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0

*Note: In Google Colab, you need to restart the runtime at this point to finalize updating the packages you just installed. **Please do not proceed to the next section without restarting.** You can also ignore the errors about version incompatibility of some of the bundled packages because we won't be using those in this notebook.*

### Check the installation

Running the code below should show the versions of the packages. Please re-run the install if you are seeing errors and don't forget to restart the runtime after re-installation.

In [3]:
# Import packages and print versions
import tensorflow as tf
import tensorflow_model_analysis as tfma
import tensorflow_data_validation as tfdv

print('TF version: {}'.format(tf.__version__))
print('TFMA version: {}'.format(tfma.__version__))
print('TFDV version: {}'.format(tfdv.__version__))

2022-11-21 15:11:17.868971: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-11-21 15:11:17.869584: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


TF version: 2.5.3
TFMA version: 0.32.0
TFDV version: 1.1.0


### Load the files

Next, you will download the files you will need for this exercise:

* Test datasets
* Data schema
* Pretrained models

We've also defined some global variables below so you can access these files throughout the notebook more easily.

In [ ]:
import os

# String variables for file and directory names
URL = 'https://storage.googleapis.com/mlep-public/course_3/week4/C3_W4_Lab_1_starter_files.tar.gz'
TAR_NAME = 'C3_W4_Lab_1_starter_files.tar.gz'
BASE_DIR = 'starter_files'
DATA_DIR = os.path.join(BASE_DIR, 'data')
CSV_DIR = os.path.join(DATA_DIR, 'csv')
TFRECORD_DIR = os.path.join(DATA_DIR, 'tfrecord')
MODELS_DIR = os.path.join(BASE_DIR, 'models')
SCHEMA_FILE = os.path.join(BASE_DIR, 'schema.pbtxt')

In [ ]:
# uncomment this line if you've downloaded the files before and want to reset
# !rm -rf {BASE_DIR}

# Download the tar file from GCP
!wget {URL}

# Extract the tar file to the base directory
!tar xzf {TAR_NAME}

# Delete tar file
!rm {TAR_NAME}

You can see the top level file and directories by running the cell below (or just using the file explorer on the left side of this Colab). We'll discuss what each contain in the next sections.

In [ ]:
print("Here's what we downloaded:")
!ls {BASE_DIR}

### Preview the dataset

The `data/csv` directory contains the test split of the Census Income dataset. We've divided it into several files for this demo notebook:

* `data_test.csv` - 15000 rows of test data
* `data_test_1.csv` - first 5000 rows of `data_test.csv`
* `data_test_2.csv` - next 5000 rows of `data_test.csv`
* `data_test_3.csv` - last 5000 rows of `data_test.csv`

You can see the description of each column [here](http://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.names) (please open link in a new window if Colab prevents the download). Also for simplicity, we've already preprocessed the `label` column as binary (i.e. `0` or `1`) to match the model's output. In your own projects, your labels might be in a different data type (e.g. string) and you want to transform that first so you can evaluate your model properly. You can preview the first few rows below:

In [ ]:
# Path to the full test set
TEST_DATA_PATH = os.path.join(CSV_DIR, 'data_test.csv')

# Preview the first few rows
!head {TEST_DATA_PATH}

### Parse the Schema

You also downloaded a schema generated by [TensorFlow Data Validation](https://www.tensorflow.org/tfx/data_validation/). You should be familiar with this file type already from previous courses. You will load it now so you can use it in the later parts of the notebook.

In [ ]:
# Load the schema as a protocol buffer
SCHEMA = tfdv.load_schema_text(SCHEMA_FILE)

# Display the schema
tfdv.display_schema(SCHEMA)

### Use the Schema to Create TFRecords

TFMA needs a TFRecord file input so you need to convert the CSVs in the data directory. If you've done the earlier labs, you will know that this can be easily done with `ExampleGen`. For this notebook however, you will use the helper function below instead to demonstrate how it can be done outside a TFX pipeline. You will pass in the schema you loaded in the previous step to determine the correct type of each feature.

In [ ]:
# imports for helper function
import csv
from tensorflow.core.example import example_pb2
from tensorflow_metadata.proto.v0 import schema_pb2

def csv_to_tfrecord(schema, csv_file, tfrecord_file):
  ''' Converts a csv file into a tfrecord
  Args:
    schema (schema_pb2) - Schema protobuf from TFDV
    csv_file (string) - file to convert to tfrecord
    tfrecord_file (string) - filename of tfrecord to create

  Returns:
    filename of tfrecord
  '''

  # Open CSV file for reading. Each row is mapped as a dictionary.
  reader = csv.DictReader(open(csv_file, 'r'))
  
  # Initialize TF examples list
  examples = []

  # For each row in CSV, create a TF Example based on
  # the Schema and append to the list
  for line in reader:

    # Intialize example
    example = example_pb2.Example()

    # Loop through features in the schema
    for feature in schema.feature:

      # Get current feature name
      key = feature.name

      # Populate values based on data type of current feature
      if feature.type == schema_pb2.FLOAT:
        example.features.feature[key].float_list.value[:] = (
            [float(line[key])] if len(line[key]) > 0 else [])
      elif feature.type == schema_pb2.INT:
        example.features.feature[key].int64_list.value[:] = (
            [int(line[key])] if len(line[key]) > 0 else [])
      elif feature.type == schema_pb2.BYTES:
        example.features.feature[key].bytes_list.value[:] = (
            [line[key].encode('utf8')] if len(line[key]) > 0 else [])
        
    # Append to the list
    examples.append(example)

  # Write examples to tfrecord file
  with tf.io.TFRecordWriter(tfrecord_file) as writer:
    for example in examples:
      writer.write(example.SerializeToString())
  
  return tfrecord_file

The code below will do the conversion and we've defined some more global variables that you will use in later exercises. 

In [ ]:
# Create tfrecord directory
!mkdir {TFRECORD_DIR}

# Create list of tfrecord files
tfrecord_files = [csv_to_tfrecord(SCHEMA, f'{CSV_DIR}/{name}', f"{TFRECORD_DIR}/{name.replace('csv','tfrecord')}") 
  for name in os.listdir(CSV_DIR)]

# Print created files
print(f'files created: {tfrecord_files}')

# Create variables for each tfrecord
TFRECORD_FULL = os.path.join(TFRECORD_DIR, 'data_test.tfrecord')
TFRECORD_DAY1 = os.path.join(TFRECORD_DIR, 'data_test_1.tfrecord')
TFRECORD_DAY2 = os.path.join(TFRECORD_DIR, 'data_test_2.tfrecord')
TFRECORD_DAY3 = os.path.join(TFRECORD_DIR, 'data_test_3.tfrecord')

# Delete unneeded variable
del tfrecord_files

### Pretrained models

Lastly, you also downloaded pretrained Keras models and they are stored in the `models/` directory. TFMA supports a number of different model types including TF Keras models, models based on generic TF2 signature APIs, as well TF estimator based models. The [get_started](https://www.tensorflow.org/tfx/model_analysis/get_started) guide has the full list of model types supported and any restrictions. You can also consult the [FAQ](https://www.tensorflow.org/tfx/model_analysis/faq) for examples on how to configure these models.

We have included three models and you can choose to analyze any one of them in the later sections. These were saved in [SavedModel](https://www.tensorflow.org/guide/saved_model) format which is the default when saving with the Keras Models API.

In [ ]:
# list model directories
!ls {MODELS_DIR}

# Create string variables for each model directory
MODEL1_FILE = os.path.join(MODELS_DIR, 'model1')
MODEL2_FILE = os.path.join(MODELS_DIR, 'model2')
MODEL3_FILE = os.path.join(MODELS_DIR, 'model3')

As mentioned earlier, these models were trained on the [Census Income dataset](http://archive.ics.uci.edu/ml/datasets/Census+Income). The label is `1` if a person earns more than 50k USD and `0` if less than or equal. You can load one of the models and look at the summary to get a sense of its architecture. All three models use the same architecture but were trained with different epochs to simulate varying performance.

In [ ]:
# Load model 1
model = tf.keras.models.load_model(MODEL1_FILE)

# Print summary. You can ignore the warnings at the start.
model.summary()

You can see the code to build these in the next lab. For now, you'll only need to take note of a few things. First, the output is a single dense unit with a sigmoid activation (i.e. `dense_5` above). This is standard for binary classification problems.

Another is that the model is exported with a transformation layer. You can see this in the summary above at the bottom row named `transform_features_layer` and it is not connected to the other layers. From previous labs, you will know that this is taken from the graph generated by the `Transform` component. It helps to avoid training-serving skews by making sure that raw inputs are transformed in the same way that the model expects. It is also available as a `tft_layer` property of the model object.

In [ ]:
# Transformation layer can be accessed in two ways. These are equivalent.
model.get_layer('transform_features_layer') is model.tft_layer

TFMA invokes this layer automatically for your raw inputs but we've included a short snippet below to demonstrate how the transformation works. You can see that the raw features are indeed reformatted to an acceptable input for the model. The raw numeric features are scaled and the raw categorical (string) features are encoded to one-hot vectors.

In [ ]:
from tensorflow_transform.tf_metadata import schema_utils

# Load one tfrecord
tfrecord_file = tf.data.TFRecordDataset(TFRECORD_DAY1)

# Parse schema object as a feature spec
feature_spec = schema_utils.schema_as_feature_spec(SCHEMA).feature_spec

# Create a batch from the dataset
for records in tfrecord_file.batch(1).take(1):

  # Parse the batch to get a dictionary of raw features
  parsed_examples = tf.io.parse_example(records, feature_spec)

  # Print the results
  print("\nRAW FEATURES:")
  for key, value in parsed_examples.items():
    print(f'{key}: {value.numpy()}')
  
  # Pop the label since the model does not expect a label input
  parsed_examples.pop('label')

  # Transform the rest of the raw features using the transform layer
  transformed_examples = model.tft_layer(parsed_examples)

  # Print the input to the model
  print("\nTRANSFORMED FEATURES:")
  for key, value in transformed_examples.items():
    print(f'{key}: {value.numpy()}')

The transformed features can be passed into the model to get the predictions. The snippet below demonstrates this and we used a low-threshold [BinaryAccuracy](https://www.tensorflow.org/api_docs/python/tf/keras/metrics/BinaryAccuracy) metric to compare the true labels and model predictions.

In [ ]:
from tensorflow_transform.tf_metadata import schema_utils

# Load one tfrecord
tfrecord_file = tf.data.TFRecordDataset(TFRECORD_DAY1)

# Parse schema object as a feature spec
feature_spec = schema_utils.schema_as_feature_spec(SCHEMA).feature_spec

# Create a batch from the dataset
for records in tfrecord_file.batch(5).take(1):

  # Get the label values from the raw input
  parsed_examples = tf.io.parse_example(records, feature_spec)
  y_true = parsed_examples.pop('label')
  print(f'labels:\n {y_true.numpy()}\n')
  
  # Transform the raw features and pass to the model to get predictions
  transformed_examples = model.tft_layer(parsed_examples)
  y_pred = model(transformed_examples)
  print(f'predictions:\n {y_pred.numpy()}\n')
  
  # Measure the binary accuracy
  metric = tf.keras.metrics.BinaryAccuracy(threshold=0.3)
  metric.update_state(y_true, y_pred)
  print(f'binary accuracy: {metric.result().numpy()}\n')

Last thing to note is the model is also exported with a [serving signature](https://www.tensorflow.org/guide/saved_model#specifying_signatures_during_export). You will know more about this in the next lab and in later parts of the specialization but for now, you can think of it as a configuration for when the model is deployed for inference. For this particular model, the default signature is configured to transform batches of serialized raw features before feeding to the model input. That way, you wouldn't have to explicitly code the transformations as previously shown in the snippet above. You can just pass in batches of data directly as shown below. 

In [ ]:
# Load one tfrecord
tfrecord_file = tf.data.TFRecordDataset(TFRECORD_DAY1)

# Print available signatures
print(f'model signatures: {model.signatures}\n')

# Create a batch
for records in tfrecord_file.batch(5).take(1):

  # Pass the batch to the model serving signature to get predictions
  output = model.signatures['serving_default'](examples=records)

  # Print results
  print(f"predictions:\n {output['output_0']}\n")

TFMA accesses this model signature so it can work with the raw data and evaluate the model to get the metrics. Not only that, it can also extract specific features and domain values from your dataset before it computes these metrics. Let's see how this is done in the next section.

## Setup and Run TFMA

With the dataset and model now available, you can now move on to use TFMA. There are some additional steps needed:

* Create a `tfma.EvalConfig` protocol message containing details about the models, metrics, and data slices you'd like to analyze
* Create a `tfma.EvalSharedModel` that points to your saved models.
* Specify an output path where the analysis results will be stored 

### Create EvalConfig

The [tfma.EvalConfig()](https://www.tensorflow.org/tfx/model_analysis/api_docs/python/tfma/EvalConfig?hl=nn&skip_cache=true) is a protocol message that sets up the analysis. Here, you will specify:

* [`model_specs`](https://www.tensorflow.org/tfx/model_analysis/api_docs/python/tfma/ModelSpec) - message containing at least the label key so it can be extracted from the evaluation/test data

* [`metrics_specs`](https://www.tensorflow.org/tfx/model_analysis/api_docs/python/tfma/MetricsSpec) - containing the metrics you would like to evaluate. A comprehensive guide can be found [here](https://www.tensorflow.org/tfx/model_analysis/metrics) and you will use the binary classification metrics for this exercise.

* [`slicing_specs`](https://www.tensorflow.org/tfx/model_analysis/api_docs/python/tfma/SlicingSpec) - containing the feature slices you would like to compute metrics for. A short description of different types of slices is shown [here](https://www.tensorflow.org/tfx/model_analysis/setup#slicing_specs)

The eval config should be passed as a protocol message and you can use the [google.protobuf.text_format](https://googleapis.dev/python/protobuf/latest/google/protobuf/text_format.html) module for that as shown below.

In [ ]:
import tensorflow_model_analysis as tfma
from google.protobuf import text_format

# Setup tfma.EvalConfig settings
eval_config = text_format.Parse("""
  ## Model information
  model_specs {
    # For keras (and serving models), you need to add a `label_key`.
    label_key: "label"
  }

  ## Post training metric information. These will be merged with any built-in
  ## metrics from training.
  metrics_specs {
    metrics { class_name: "ExampleCount" }
    metrics { class_name: "BinaryAccuracy" }
    metrics { class_name: "BinaryCrossentropy" }
    metrics { class_name: "AUC" }
    metrics { class_name: "AUCPrecisionRecall" }
    metrics { class_name: "Precision" }
    metrics { class_name: "Recall" }
    metrics { class_name: "MeanLabel" }
    metrics { class_name: "MeanPrediction" }
    metrics { class_name: "Calibration" }
    metrics { class_name: "CalibrationPlot" }
    metrics { class_name: "ConfusionMatrixPlot" }
    # ... add additional metrics and plots ...
  }

  ## Slicing information

  # overall slice
  slicing_specs {}

  # slice specific features
  slicing_specs {
    feature_keys: ["sex"]
  }
  slicing_specs {
    feature_keys: ["race"]
  }

  # slice specific values from features
  slicing_specs {
    feature_values: {
      key: "native-country"
      value: "Cambodia"
    }
  }
  slicing_specs {
    feature_values: {
      key: "native-country"
      value: "Canada"
    }
  }

  # slice feature crosses
  slicing_specs {
    feature_keys: ["sex", "race"]
  }
""", tfma.EvalConfig())

### Create EvalSharedModel

TFMA also requires an [EvalSharedModel](https://www.tensorflow.org/tfx/model_analysis/api_docs/python/tfma/types/EvalSharedModel) instance that points to your model so it can be shared between multiple threads in the same process. This instance includes information about the type of model (keras, etc) and how to load and configure the model from its saved location on disk (e.g. tags, etc). The [tfma.default_eval_shared_model()](https://www.tensorflow.org/tfx/model_analysis/api_docs/python/tfma/default_eval_shared_model) API can be used to create this given the model location and eval config.

In [ ]:
# Create a tfma.EvalSharedModel that points to your model.
# You can ignore the warnings generated.
eval_shared_model = tfma.default_eval_shared_model(
    eval_saved_model_path=MODEL1_FILE,
    eval_config=eval_config)

In [ ]:
# Show properties of EvalSharedModel
print(f'EvalSharedModel contents: {eval_shared_model}')

### Run TFMA

With the setup complete, you just need to declare an output directory then run TFMA. You will pass in the eval config, shared model, dataset, and output directory to [`tfma.run_model_analysis()`](https://www.tensorflow.org/tfx/model_analysis/api_docs/python/tfma/run_model_analysis) as shown below. This will create a [`tfma.EvalResult`](https://www.tensorflow.org/tfx/model_analysis/api_docs/python/tfma/EvalResult) which you can use later for rendering metrics and plots.

In [ ]:
# Specify output path for the evaluation results
OUTPUT_DIR = os.path.join(BASE_DIR, 'output')

# Run TFMA. You can ignore the warnings generated.
eval_result = tfma.run_model_analysis(
    eval_shared_model=eval_shared_model,
    eval_config=eval_config,
    data_location=TFRECORD_FULL,
    output_path=OUTPUT_DIR)

## Visualizing Metrics and Plots

You can visualize the results also using TFMA methods. In this section, you will view the returned metrics and plots for the different slices you specified in the eval config.

### Rendering Metrics

You can view the metrics with the [`tfma.view.render_slicing_metrics()`](https://www.tensorflow.org/tfx/model_analysis/api_docs/python/tfma/view/render_slicing_metrics) method. By default, the views will display the `Overall` slice. To view a particular slice you can pass in a feature name to the `slicing_column` argument as shown below. You can visualize the different metrics through the `Show` dropdown menu and you can hover over the bar charts to show the exact value measured. 

We encourage you to try the different options you see and also modify the command. Here are some examples:

* Removing the `slicing_column` argument will produce the overall slice.
* You can also pass in `race` (since it was specified in the eval config) to see the results for that particular slice.
* Using the `Examples (Weighted) Threshold` slider above 5421 will remove the `Female` slice because it has less examples than that.
* Toggling the `View` dropdown to `Metrics Histogram` will show the results divided into buckets. For example, if you're slicing column is `sex` and the `Histogram Type` dropdown is at `Slice Counts`, then you will one slice in two of the 10 (default) buckets since we only have two values for that feature ('Male' and 'Female'). The x-axis show the values for the metric in the `Select Metric` dropdown. This is the default view when the number of slices is large.
* At the bottom of the screen, you will notice the measurements also presented in tabular format. You can sort it by clicking on the feature name headers.

In [ ]:
# Render metrics for a feature
tfma.view.render_slicing_metrics(eval_result, slicing_column='sex')

### More Slices

If you haven't yet, you can also pass in the `native-country` to the slicing column. The difference in this visualization is we only specified two of its values in the eval config earlier. This is useful if you just want to study a subgroup of a particular feature and not the entire domain.

In [ ]:
# Render metrics for feature. Review EvalConfig message to see what values were selected.
tfma.view.render_slicing_metrics(eval_result, slicing_column='native-country')

TFMA also supports creating feature crosses to analyze combinations of features. Our original settings created a cross between `sex` and `race` and you can pass it in as a [SlicingSpec](https://www.tensorflow.org/tfx/model_analysis/api_docs/python/tfma/SlicingSpec) as shown below.

In [ ]:
# Render metrics for feature crosses
tfma.view.render_slicing_metrics(
    eval_result,
    slicing_spec=tfma.SlicingSpec(
        feature_keys=['sex', 'race']))

In some cases, crossing the two columns creates a lot of combinations. You can narrow down the results to only look at specific values by specifying it in the `slicing_spec` argument. Below shows the results for the `sex` feature for the `Other` race.

In [ ]:
# Narrow down the feature crosses by specifying feature values
tfma.view.render_slicing_metrics(
    eval_result,
    slicing_spec=tfma.SlicingSpec(
        feature_keys=['sex'], feature_values={'race': 'Other'}))

### Rendering Plots

Any plots that were added to the `tfma.EvalConfig` as post training `metric_specs` can be displayed using [`tfma.view.render_plot`](https://www.tensorflow.org/tfx/model_analysis/api_docs/python/tfma/view/render_plot).

As with metrics, plots can be viewed by slice. Unlike metrics, only plots for a particular slice value can be displayed so the `tfma.SlicingSpec` must be used and it must specify both a slice feature name and value. If no slice is provided then the plots for the `Overall` slice is used.

The example below displays the plots that were computed for the `sex:Male` slice. You can click on the names at the bottom of the graph to see a different plot type. Alternatively, you can tick the `Show all plots` checkbox to show all the plots in one screen.

In [ ]:
# Render plots
tfma.view.render_plot(
    eval_result,
    tfma.SlicingSpec(feature_values={'sex': 'Male'}))

## Tracking Model Performance Over Time

Your training dataset will be used for training your model, and will hopefully be representative of your test dataset and the data that will be sent to your model in production.  However, while the data in inference requests may remain the same as your training data, it can also start to change enough so that the performance of your model will change. That means that you need to monitor and measure your model's performance on an ongoing basis so that you can be aware of and react to changes.

Let's take a look at how TFMA can help. You will load three different datasets and compare the model analysis results using the [`render_time_series()`](https://www.tensorflow.org/tfx/model_analysis/api_docs/python/tfma/view/render_time_series) method.

In [ ]:
import os

# Put data paths we prepared earlier in a list
TFRECORDS = [TFRECORD_DAY1, TFRECORD_DAY2, TFRECORD_DAY3]

# Initialize output paths list for each result 
output_paths = []

# Run eval on each tfrecord separately
for num, tfrecord in enumerate(TFRECORDS):

  # Use the same model as before
  eval_shared_model = tfma.default_eval_shared_model(
      eval_saved_model_path=MODEL1_FILE,
      eval_config=eval_config)

  # Prepare output path name
  output_path = os.path.join('.', 'time_series', str(num))
  output_paths.append(output_path)

  # Run TFMA on the current tfrecord in the loop
  tfma.run_model_analysis(eval_shared_model=eval_shared_model,
                          eval_config=eval_config,
                          data_location=tfrecord,
                          output_path=output_path)

First, imagine that you've trained and deployed your model yesterday. And now, you want to see how it's doing on the new data coming in today.  The visualization will start by displaying AUC. From the UI, you can:

* Add other metrics using the "Add metric series" menu.
* Close unwanted graphs by clicking on x
* Hover over data points (the ends of line segments in the graph) to get more details

Note: In the metric series charts, the x-axis is just the model directory name of the model that you're examining.

In [ ]:
# Load results for day 1 and day 2 datasets
eval_results_from_disk = tfma.load_eval_results(output_paths[:2])

# Visualize results
tfma.view.render_time_series(eval_results_from_disk)

Now imagine that another day has passed and you want to see how it's doing on the new data coming in today.

In [ ]:
# Load results for all three days
eval_results_from_disk = tfma.load_eval_results(output_paths)

# Visualize the results
tfma.view.render_time_series(eval_results_from_disk)

This type of investigation lets you see if your model is behaving poorly on new data. You can make the decision to retrain your production model based on these results. Retraining might not always produce the best results and you also need a way to detect that. You will see how TFMA helps in that regard in the next section.

## Model Validation

TFMA can be configured to evaluate multiple models at the same time. Typically, this is done to compare a candidate model against a baseline (such as the currently serving model) to determine what the performance differences in metrics are. When [thresholds](https://www.tensorflow.org/tfx/model_analysis/api_docs/python/tfma/MetricThreshold) are configured, TFMA will produce a [`tfma.ValidationResult`](https://www.tensorflow.org/tfx/model_analysis/api_docs/python/tfma/ValidationResult) record indicating whether the performance matches expecations.

Below, you will re-configure the EvalConfig settings to compare two models: a candidate and a baseline. You will also validate the candidate's performance against the baseline by setting a [`tmfa.MetricThreshold`](https://www.tensorflow.org/tfx/model_analysis/api_docs/python/tfma/MetricThreshold) on the `BinaryAccuracy` metric. This helps in determining if your new model can indeed replace your currently deployed model.

In [ ]:
# Setup tfma.EvalConfig setting with metric thresholds
eval_config_with_thresholds = text_format.Parse("""
  ## Model information
  model_specs {
    name: "candidate"
    label_key: "label"
  }
  model_specs {
    name: "baseline"
    label_key: "label"
    is_baseline: true
  }

  ## Post training metric information
  metrics_specs {
    metrics { class_name: "ExampleCount" }
    metrics {
      class_name: "BinaryAccuracy"
      threshold {
        # Ensure that metric is always > 0.9
        value_threshold {
          lower_bound { value: 0.9 }
        }
        # Ensure that metric does not drop by more than a small epsilon
        # e.g. (candidate - baseline) > -1e-10 or candidate > baseline - 1e-10
        change_threshold {
          direction: HIGHER_IS_BETTER
          absolute { value: -1e-10 }
        }
      }
    }
    metrics { class_name: "BinaryCrossentropy" }
    metrics { class_name: "AUC" }
    metrics { class_name: "AUCPrecisionRecall" }
    metrics { class_name: "Precision" }
    metrics { class_name: "Recall" }
    metrics { class_name: "MeanLabel" }
    metrics { class_name: "MeanPrediction" }
    metrics { class_name: "Calibration" }
    metrics { class_name: "CalibrationPlot" }
    metrics { class_name: "ConfusionMatrixPlot" }
    # ... add additional metrics and plots ...
  }

  ## Slicing information
  slicing_specs {}  # overall slice
  slicing_specs {
    feature_keys: ["race"]
  }
  slicing_specs {
    feature_keys: ["sex"]
  }
""", tfma.EvalConfig())

# Create tfma.EvalSharedModels that points to the candidate and baseline
candidate_model_path = MODEL1_FILE
baseline_model_path = MODEL2_FILE

eval_shared_models = [
  tfma.default_eval_shared_model(
      model_name=tfma.CANDIDATE_KEY,
      eval_saved_model_path=candidate_model_path,
      eval_config=eval_config_with_thresholds),
  tfma.default_eval_shared_model(
      model_name=tfma.BASELINE_KEY,
      eval_saved_model_path=baseline_model_path,
      eval_config=eval_config_with_thresholds),
]

# Specify validation path
validation_output_path = os.path.join(OUTPUT_DIR, 'validation')

# Run TFMA on the two models
eval_result_with_validation = tfma.run_model_analysis(
    eval_shared_models,
    eval_config=eval_config_with_thresholds,
    data_location=TFRECORD_FULL,
    output_path=validation_output_path)

When running evaluations with one or more models against a baseline, TFMA automatically adds different metrics for all the metrics computed during the evaluation. These metrics are named after the corresponding metric but with the string `_diff` appended to the metric name. A positive value for these `_diff` metrics indicates an improved performance against the baseline.

Like in the previous section, you can view the results with `render_time_series()`.

In [ ]:
# Render results
tfma.view.render_time_series(eval_result_with_validation)

You can use [`tfma.load_validator_result`](https://www.tensorflow.org/tfx/model_analysis/api_docs/python/tfma/load_validation_result) to view the validation results you specified with the threshold settings. For this example, the validation fails because `BinaryAccuracy` is below the threshold.

In [ ]:
# Print validation result
validation_result = tfma.load_validation_result(validation_output_path)
print(validation_result.validation_ok)

**Congratulations! You have now explored the different methods of model analysis using TFMA. In the next section, you will see how these can fit into a TFX pipeline so you can automate the process and store the results in your pipeline directory and metadata store.**